In [18]:
"""
Seasonal Demand Trends for Luxury Brands Analysis
------------------------------------------------
This script analyzes how demand fluctuates across different seasons for luxury brands
on the Vestiaire platform.

Author: Samarth
Date: February 25, 2025
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
import os

# Suppress warnings
warnings.filterwarnings('ignore')

class SeasonalLuxuryDemandAnalyzer:
    """Class for analyzing seasonal demand trends for luxury brands."""

    def __init__(self, config=None):
        """Initialize the analyzer with configuration parameters."""
        # Default configuration
        self.default_config = {
            'data': {
                'sample_size': 900515,  # Number of rows to sample from the dataset
            },
            'brand_tiers': {
                'ultra_luxury': [
                    'chanel', 'hermes', 'louis vuitton', 'gucci', 'prada', 'dior',
                    'cartier', 'bulgari', 'fendi', 'balenciaga', 'bottega veneta'
                ],
                'luxury': [
                    'burberry', 'versace', 'armani', 'valentino', 'givenchy', 'ysl',
                    'saint laurent', 'celine', 'dolce & gabbana', 'tiffany',
                    'miu miu', 'loewe', 'balmain', 'alexander mcqueen'
                ],
                'premium': [
                    'michael kors', 'coach', 'kate spade', 'tory burch', 'calvin klein',
                    'tommy hilfiger', 'marc jacobs', 'hugo boss', 'ralph lauren',
                    'jimmy choo', 'acne studios', 'stella mccartney', 'kenzo'
                ]
            },
            'season_mapping': {
                'Autumn / Winter': 'Fall/Winter',
                'Spring / Summer': 'Spring/Summer',
                'All seasons': 'All Seasons'
            },
            'plots': {
                'figsize': (12, 8),
                'style': 'whitegrid',
                'palette': 'viridis',
                'dpi': 100
            },
            'output': {
                'figures_dir': 'figures/',
                'data_dir': 'data/'
            }
        }

        # Override defaults with provided config
        if config:
            self.config = self._update_config(self.default_config, config)
        else:
            self.config = self.default_config

        # Initialize class attributes
        self.data = None
        self.seasonal_stats = None

    def _update_config(self, default_config, new_config):
        """Recursively update the default config with new values."""
        config = default_config.copy()

        for key, value in new_config.items():
            if key in config and isinstance(value, dict) and isinstance(config[key], dict):
                config[key] = self._update_config(config[key], value)
            else:
                config[key] = value

        return config

    def load_data(self, filepath, encoding='utf-8'):
        """Load the dataset from a CSV file."""
        print(f"Loading data from {filepath}...")

        # Read the data
        data = pd.read_csv(filepath, encoding=encoding)

        # Sample data if needed
        if self.config['data']['sample_size'] and self.config['data']['sample_size'] < len(data):
            data = data.sample(n=self.config['data']['sample_size'], random_state=42)

        self.data = data
        print(f"Data loaded. Shape: {data.shape}")

        return self

    def preprocess_data(self):
        """Preprocess the data for analysis."""
        if self.data is None:
            raise ValueError("Data not loaded. Please call load_data() first.")

        print("\n==== Preprocessing Data ====")

        # Make a copy of the data to avoid modifying the original
        df = self.data.copy()

        # Handle missing values
        for col in ['product_season', 'brand_name', 'product_like_count', 'price_usd', 'sold']:
            if col in df.columns and df[col].isna().any():
                if col in ['product_like_count', 'price_usd']:
                    # For numeric columns, fill with median
                    df[col] = df[col].fillna(df[col].median())
                elif col in ['product_season', 'brand_name']:
                    # For categorical columns, fill with most frequent
                    df[col] = df[col].fillna(df[col].mode()[0])
                elif col == 'sold':
                    # For target, fill with False
                    df[col] = df[col].fillna(False)

        # Standardize season values using the mapping
        if 'product_season' in df.columns:
            df['season_standardized'] = df['product_season'].map(
                self.config['season_mapping']).fillna(df['product_season'])

        # Convert boolean values to integers if needed
        if 'sold' in df.columns and df['sold'].dtype == bool:
            df['sold'] = df['sold'].astype(int)

        # Add brand tier column
        if 'brand_name' in df.columns:
            df['brand_tier'] = df['brand_name'].apply(self._categorize_brand_tier)

        # Update the data
        self.data = df

        print(f"Preprocessing complete. Data shape: {df.shape}")
        return self

    def _categorize_brand_tier(self, brand_name):
        """Categorize brands into tiers based on luxury level."""
        if not isinstance(brand_name, str):
            return 'unknown'

        brand_lower = brand_name.lower()

        for tier, brands in self.config['brand_tiers'].items():
            if any(brand in brand_lower for brand in brands):
                return tier

        return 'standard'

    def analyze_seasonal_demand(self):
        """Analyze seasonal demand trends for luxury brands."""
        if self.data is None:
            raise ValueError("Data not loaded. Please call load_data() first.")

        print("\n==== Analyzing Seasonal Demand ====")

        df = self.data

        # Group by season and brand tier
        grouped = df.groupby(['season_standardized', 'brand_tier'])

        # Calculate statistics
        seasonal_stats = grouped.agg({
            'sold': ['count', 'sum', 'mean'],
            'price_usd': ['mean', 'median', 'std'],
            'product_like_count': ['mean', 'median', 'sum']
        }).reset_index()

        # Flatten multi-level columns
        seasonal_stats.columns = ['_'.join(col).strip('_') if isinstance(col, tuple) else col for col in seasonal_stats.columns]

        # Calculate additional metrics
        seasonal_stats['demand_index'] = seasonal_stats['product_like_count_sum'] / seasonal_stats['sold_count']
        seasonal_stats['conversion_rate'] = (seasonal_stats['sold_mean'] * 100).round(2)

        # Save statistics
        self.seasonal_stats = seasonal_stats

        print("Seasonal demand analysis complete.")
        return self

    def visualize_trends(self):
        """Create visualizations of seasonal demand trends."""
        if self.seasonal_stats is None:
            raise ValueError("Seasonal stats not calculated. Please call analyze_seasonal_demand() first.")

        print("\n==== Creating Visualizations ====")

        # Set plot style
        sns.set(style=self.config['plots']['style'])

        # Create output directory if it doesn't exist
        os.makedirs(self.config['output']['figures_dir'], exist_ok=True)

        # 1. Sales percentage by season and brand tier
        self._plot_sales_percentage()

        # 2. Average price by season and brand tier
        self._plot_average_price()

        # 3. Average likes by season and brand tier
        self._plot_average_likes()

        # 4. Conversion rate by season and brand tier
        self._plot_conversion_rate()

        # 5. Heatmap of seasonal demand
        self._plot_demand_heatmap()

        print("Visualizations created successfully.")
        return self

    def _plot_sales_percentage(self):
        """Plot sales percentage by season and brand tier."""
        # Pivot data for plotting
        plot_data = self.seasonal_stats.pivot(
            index='season_standardized',
            columns='brand_tier',
            values='sold_mean'
        ).fillna(0) * 100  # Convert to percentage

        # Create figure and plot
        fig, ax = plt.subplots(figsize=self.config['plots']['figsize'], dpi=self.config['plots']['dpi'])
        plot_data.plot(kind='bar', ax=ax)

        # Set labels and title
        ax.set_title('Sales Percentage by Season and Brand Tier', fontsize=16)
        ax.set_xlabel('Season', fontsize=14)
        ax.set_ylabel('Sales Percentage (%)', fontsize=14)
        plt.setp(ax.get_xticklabels(), rotation=45)
        ax.legend(title='Brand Tier')

        # Add value labels
        for container in ax.containers:
            ax.bar_label(container, fmt='%.1f%%', fontsize=10)

        # Adjust layout and save
        fig.tight_layout()
        fig.savefig(os.path.join(self.config['output']['figures_dir'], 'sales_percentage_by_season.png'))
        plt.close(fig)

    def _plot_average_price(self):
        """Plot average price by season and brand tier."""
        # Pivot data for plotting
        plot_data = self.seasonal_stats.pivot(
            index='season_standardized',
            columns='brand_tier',
            values='price_usd_mean'
        ).fillna(0)

        # Create figure and plot
        fig, ax = plt.subplots(figsize=self.config['plots']['figsize'], dpi=self.config['plots']['dpi'])
        plot_data.plot(kind='bar', ax=ax)

        # Set labels and title
        ax.set_title('Average Price by Season and Brand Tier', fontsize=16)
        ax.set_xlabel('Season', fontsize=14)
        ax.set_ylabel('Average Price (USD)', fontsize=14)
        plt.setp(ax.get_xticklabels(), rotation=45)
        ax.legend(title='Brand Tier')

        # Add value labels
        for container in ax.containers:
            ax.bar_label(container, fmt='$%.0f', fontsize=10)

        # Adjust layout and save
        fig.tight_layout()
        fig.savefig(os.path.join(self.config['output']['figures_dir'], 'average_price_by_season.png'))
        plt.close(fig)

    def _plot_average_likes(self):
        """Plot average likes by season and brand tier."""
        # Pivot data for plotting
        plot_data = self.seasonal_stats.pivot(
            index='season_standardized',
            columns='brand_tier',
            values='product_like_count_mean'
        ).fillna(0)

        # Create figure and plot
        fig, ax = plt.subplots(figsize=self.config['plots']['figsize'], dpi=self.config['plots']['dpi'])
        plot_data.plot(kind='bar', ax=ax)

        # Set labels and title
        ax.set_title('Average Likes by Season and Brand Tier', fontsize=16)
        ax.set_xlabel('Season', fontsize=14)
        ax.set_ylabel('Average Likes', fontsize=14)
        plt.setp(ax.get_xticklabels(), rotation=45)
        ax.legend(title='Brand Tier')

        # Add value labels
        for container in ax.containers:
            ax.bar_label(container, fmt='%.1f', fontsize=10)

        # Adjust layout and save
        fig.tight_layout()
        fig.savefig(os.path.join(self.config['output']['figures_dir'], 'average_likes_by_season.png'))
        plt.close(fig)

    def _plot_conversion_rate(self):
        """Plot conversion rate by season and brand tier."""
        # Pivot data for plotting
        plot_data = self.seasonal_stats.pivot(
            index='season_standardized',
            columns='brand_tier',
            values='conversion_rate'
        ).fillna(0)

        # Create figure and plot
        fig, ax = plt.subplots(figsize=self.config['plots']['figsize'], dpi=self.config['plots']['dpi'])
        plot_data.plot(kind='bar', ax=ax)

        # Set labels and title
        ax.set_title('Conversion Rate by Season and Brand Tier', fontsize=16)
        ax.set_xlabel('Season', fontsize=14)
        ax.set_ylabel('Conversion Rate (%)', fontsize=14)
        plt.setp(ax.get_xticklabels(), rotation=45)
        ax.legend(title='Brand Tier')

        # Add value labels
        for container in ax.containers:
            ax.bar_label(container, fmt='%.1f%%', fontsize=10)

        # Adjust layout and save
        fig.tight_layout()
        fig.savefig(os.path.join(self.config['output']['figures_dir'], 'conversion_rate_by_season.png'))
        plt.close(fig)

    def _plot_demand_heatmap(self):
        """Create a heatmap of demand index by season and brand tier."""
        # Pivot data for plotting
        plot_data = self.seasonal_stats.pivot(
            index='season_standardized',
            columns='brand_tier',
            values='demand_index'
        ).fillna(0)

        # Create figure and axes
        fig, ax = plt.subplots(figsize=self.config['plots']['figsize'], dpi=self.config['plots']['dpi'])

        # Create heatmap
        sns.heatmap(plot_data, annot=True, fmt='.1f', cmap='viridis', ax=ax)

        # Set labels and title
        ax.set_title('Demand Index by Season and Brand Tier', fontsize=16)
        ax.set_xlabel('Brand Tier', fontsize=14)
        ax.set_ylabel('Season', fontsize=14)

        # Adjust layout and save
        fig.tight_layout()
        fig.savefig(os.path.join(self.config['output']['figures_dir'], 'demand_heatmap.png'))
        plt.close(fig)

    def generate_report(self):
        """Generate a summary report of seasonal demand trends."""
        if self.seasonal_stats is None:
            raise ValueError("Seasonal stats not calculated. Please call analyze_seasonal_demand() first.")

        print("\n==== Generating Report ====")

        # Create output directory if it doesn't exist
        os.makedirs(self.config['output']['data_dir'], exist_ok=True)

        # Save seasonal stats to CSV
        output_file = os.path.join(self.config['output']['data_dir'], 'seasonal_luxury_demand_stats.csv')
        self.seasonal_stats.to_csv(output_file, index=False)

        # Find best-selling seasons for each brand tier
        best_selling = {}
        for tier in self.seasonal_stats['brand_tier'].unique():
            tier_data = self.seasonal_stats[self.seasonal_stats['brand_tier'] == tier]
            best_season = tier_data.loc[tier_data['conversion_rate'].idxmax()]
            best_selling[tier] = {
                'season': best_season['season_standardized'],
                'conversion_rate': best_season['conversion_rate'],
                'avg_price': best_season['price_usd_mean'],
                'avg_likes': best_season['product_like_count_mean']
            }

        # Print summary
        print("\nSeasonal Demand Trends Summary:")
        print("---------------------------------")
        for tier, data in best_selling.items():
            print(f"\n{tier.title()} Brands:")
            print(f"  Best-selling season: {data['season']}")
            print(f"  Conversion rate: {data['conversion_rate']:.2f}%")
            print(f"  Average price: ${data['avg_price']:.2f}")
            print(f"  Average likes: {data['avg_likes']:.2f}")

        print(f"\nDetailed statistics saved to {output_file}")
        return self

    def run_analysis(self, filepath):
        """Run the complete seasonal demand analysis pipeline."""
        return (
            self.load_data(filepath)
            .preprocess_data()
            .analyze_seasonal_demand()
            .visualize_trends()
            .generate_report()
        )


def main():
    """Main function to run the seasonal luxury demand analysis."""
    # Initialize analyzer
    analyzer = SeasonalLuxuryDemandAnalyzer()

    # Run the analysis
    analyzer.run_analysis('/Users/samarthbasavarajannigeri/Downloads/vestiaire.csv')

    print("\n==== Analysis Complete ====")
    print("Seasonal demand trends for luxury brands have been analyzed and visualized.")


if __name__ == "__main__":
    main()

Loading data from /Users/samarthbasavarajannigeri/Downloads/vestiaire.csv...
Data loaded. Shape: (900514, 36)

==== Preprocessing Data ====
Preprocessing complete. Data shape: (900514, 38)

==== Analyzing Seasonal Demand ====
Seasonal demand analysis complete.

==== Creating Visualizations ====
Visualizations created successfully.

==== Generating Report ====

Seasonal Demand Trends Summary:
---------------------------------

Luxury Brands:
  Best-selling season: Spring/Summer
  Conversion rate: 2.98%
  Average price: $351.47
  Average likes: 7.69

Premium Brands:
  Best-selling season: Spring/Summer
  Conversion rate: 2.04%
  Average price: $171.16
  Average likes: 4.37

Standard Brands:
  Best-selling season: Spring/Summer
  Conversion rate: 2.38%
  Average price: $254.20
  Average likes: 5.59

Ultra_Luxury Brands:
  Best-selling season: Spring/Summer
  Conversion rate: 3.74%
  Average price: $590.69
  Average likes: 11.25

Detailed statistics saved to data/seasonal_luxury_demand_sta

In [19]:
"""
Best-Selling Brands and Products by Country Analysis
--------------------------------------------------
This script analyzes the Vestiaire dataset to identify which brands and product types
sell best in different countries, with comprehensive visualizations and insights.

Author: Samarth
Date: February 25, 2025
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import os
import warnings
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.ticker as mtick

# Suppress warnings
warnings.filterwarnings('ignore')


class CountryBrandAnalyzer:
    """Class for analyzing best-selling brands and products by country."""

    def __init__(self, config=None):
        """Initialize the analyzer with configuration parameters."""
        # Default configuration
        self.default_config = {
            'data': {
                'sample_size': 900515,  # Number of rows to sample from the dataset
                'min_listings_per_country': 300,  # Minimum number of listings for a country to be included
                'min_listings_per_brand': 50,  # Minimum number of listings for a brand to be included
                'min_listings_per_product_type': 30,  # Minimum number of listings for a product type to be included
                'top_n_brands': 20,  # Number of top brands to analyze per country
                'top_n_product_types': 15,  # Number of top product types to analyze per country
                'top_n_countries': 10,  # Number of top countries to include in the analysis
            },
            'plots': {
                'figsize_large': (14, 10),
                'figsize_medium': (12, 8),
                'figsize_small': (10, 6),
                'style': 'whitegrid',
                'palette': 'viridis',
                'cmap': 'viridis',
                'dpi': 100,
                'title_fontsize': 16,
                'label_fontsize': 12,
                'legend_fontsize': 10,
                'annotation_fontsize': 9
            },
            'output': {
                'figures_dir': 'figures/',
                'data_dir': 'data/',
            }
        }

        # Override defaults with provided config
        if config:
            self.config = self._update_config(self.default_config, config)
        else:
            self.config = self.default_config

        # Initialize class attributes
        self.data = None
        self.country_brand_stats = None
        self.country_product_stats = None
        self.overall_stats = None

    def _update_config(self, default_config, new_config):
        """Recursively update the default config with new values."""
        config = default_config.copy()

        for key, value in new_config.items():
            if key in config and isinstance(value, dict) and isinstance(config[key], dict):
                config[key] = self._update_config(config[key], value)
            else:
                config[key] = value

        return config

    def load_data(self, filepath, encoding='utf-8'):
        """Load the dataset from a CSV file."""
        print(f"Loading data from {filepath}...")

        # Read the data
        data = pd.read_csv(filepath, encoding=encoding)

        # Sample data if needed
        if self.config['data']['sample_size'] and self.config['data']['sample_size'] < len(data):
            data = data.sample(n=self.config['data']['sample_size'], random_state=42)

        self.data = data
        print(f"Data loaded. Shape: {data.shape}")

        return self

    def preprocess_data(self):
        """Preprocess the data for analysis."""
        if self.data is None:
            raise ValueError("Data not loaded. Please call load_data() first.")

        print("\n==== Preprocessing Data ====")

        # Make a copy of the data to avoid modifying the original
        df = self.data.copy()

        # Handle missing values
        for col in ['brand_name', 'product_type', 'seller_country', 'price_usd', 'sold']:
            if col in df.columns and df[col].isna().any():
                if col in ['price_usd']:
                    # For numeric columns, fill with median
                    df[col] = df[col].fillna(df[col].median())
                elif col in ['brand_name', 'product_type', 'seller_country']:
                    # For categorical columns, fill with 'Unknown'
                    df[col] = df[col].fillna('Unknown')
                elif col == 'sold':
                    # For target, fill with False
                    df[col] = df[col].fillna(False)

        # Ensure sold is boolean or int
        if 'sold' in df.columns:
            if df['sold'].dtype == bool:
                df['sold'] = df['sold'].astype(int)
            else:
                # If it's not already boolean, convert to 0/1
                df['sold'] = df['sold'].astype(int)

        # Remove any rows with missing key values after filling
        required_columns = ['brand_name', 'product_type', 'seller_country', 'sold']
        df = df.dropna(subset=required_columns)

        # Clean country names (optional, may not be needed depending on the data)
        df['seller_country'] = df['seller_country'].str.strip()

        # Clean brand names
        df['brand_name'] = df['brand_name'].str.strip()

        # Clean product types
        df['product_type'] = df['product_type'].str.strip()

        # Update the data
        self.data = df

        print(f"Preprocessing complete. Data shape: {df.shape}")
        return self

    def analyze_brands_by_country(self):
        """Analyze best-selling brands by country."""
        if self.data is None:
            raise ValueError("Data not loaded. Please call load_data() first.")

        print("\n==== Analyzing Brands by Country ====")

        df = self.data

        # Get the top countries by listing count
        country_counts = df['seller_country'].value_counts()
        top_countries = country_counts[country_counts >= self.config['data']['min_listings_per_country']].head(
            self.config['data']['top_n_countries']).index.tolist()

        print(f"Analyzing data for {len(top_countries)} countries with at least "
              f"{self.config['data']['min_listings_per_country']} listings")

        # Filter for those countries
        df_filtered = df[df['seller_country'].isin(top_countries)]

        # Calculate metrics grouped by country and brand
        country_brand_stats = []

        for country in top_countries:
            country_data = df[df['seller_country'] == country]

            # Get top brands by listing count
            brand_counts = country_data['brand_name'].value_counts()
            top_brands = brand_counts[brand_counts >= self.config['data']['min_listings_per_brand']].head(
                self.config['data']['top_n_brands']).index.tolist()

            # For each top brand, calculate metrics
            for brand in top_brands:
                brand_data = country_data[country_data['brand_name'] == brand]

                # Calculate metrics
                listings = len(brand_data)
                sales = brand_data['sold'].sum()
                sales_rate = sales / listings * 100 if listings > 0 else 0
                avg_price = brand_data['price_usd'].mean()

                # Store results
                country_brand_stats.append({
                    'country': country,
                    'brand': brand,
                    'listings': listings,
                    'sales': sales,
                    'sales_rate': sales_rate,
                    'avg_price': avg_price
                })

        # Convert to DataFrame
        self.country_brand_stats = pd.DataFrame(country_brand_stats)

        print(f"Brand analysis complete. Analyzed {len(self.country_brand_stats)} country-brand combinations.")
        return self

    def analyze_products_by_country(self):
        """Analyze best-selling product types by country."""
        if self.data is None:
            raise ValueError("Data not loaded. Please call load_data() first.")

        print("\n==== Analyzing Product Types by Country ====")

        df = self.data

        # Get the top countries by listing count (use the same ones as in brand analysis)
        country_counts = df['seller_country'].value_counts()
        top_countries = country_counts[country_counts >= self.config['data']['min_listings_per_country']].head(
            self.config['data']['top_n_countries']).index.tolist()

        # Calculate metrics grouped by country and product type
        country_product_stats = []

        for country in top_countries:
            country_data = df[df['seller_country'] == country]

            # Get top product types by listing count
            product_counts = country_data['product_type'].value_counts()
            top_products = product_counts[product_counts >= self.config['data']['min_listings_per_product_type']].head(
                self.config['data']['top_n_product_types']).index.tolist()

            # For each top product type, calculate metrics
            for product_type in top_products:
                product_data = country_data[country_data['product_type'] == product_type]

                # Calculate metrics
                listings = len(product_data)
                sales = product_data['sold'].sum()
                sales_rate = sales / listings * 100 if listings > 0 else 0
                avg_price = product_data['price_usd'].mean()

                # Store results
                country_product_stats.append({
                    'country': country,
                    'product_type': product_type,
                    'listings': listings,
                    'sales': sales,
                    'sales_rate': sales_rate,
                    'avg_price': avg_price
                })

        # Convert to DataFrame
        self.country_product_stats = pd.DataFrame(country_product_stats)

        print(f"Product type analysis complete. Analyzed {len(self.country_product_stats)} country-product combinations.")
        return self

    def calculate_overall_statistics(self):
        """Calculate overall statistics across countries."""
        if self.country_brand_stats is None or self.country_product_stats is None:
            raise ValueError("Brand and product analyses not performed. Please run analyze_brands_by_country() and analyze_products_by_country() first.")

        print("\n==== Calculating Overall Statistics ====")

        # Get top countries (should be the same in both analyses)
        top_countries = self.country_brand_stats['country'].unique()

        # Calculate country-level statistics
        country_stats = {}

        for country in top_countries:
            country_brands = self.country_brand_stats[self.country_brand_stats['country'] == country]
            country_products = self.country_product_stats[self.country_product_stats['country'] == country]

            # Top selling brand by sales rate
            if not country_brands.empty:
                top_brand_by_rate = country_brands.loc[country_brands['sales_rate'].idxmax()]
                top_brand_by_volume = country_brands.loc[country_brands['sales'].idxmax()]
            else:
                top_brand_by_rate = None
                top_brand_by_volume = None

            # Top selling product type by sales rate
            if not country_products.empty:
                top_product_by_rate = country_products.loc[country_products['sales_rate'].idxmax()]
                top_product_by_volume = country_products.loc[country_products['sales'].idxmax()]
            else:
                top_product_by_rate = None
                top_product_by_volume = None

            # Store results
            country_stats[country] = {
                'top_brand_by_rate': top_brand_by_rate,
                'top_brand_by_volume': top_brand_by_volume,
                'top_product_by_rate': top_product_by_rate,
                'top_product_by_volume': top_product_by_volume,
                'total_listings': self.data[self.data['seller_country'] == country].shape[0],
                'total_sales': self.data[(self.data['seller_country'] == country) & (self.data['sold'] == 1)].shape[0],
                'overall_sales_rate': self.data[(self.data['seller_country'] == country) & (self.data['sold'] == 1)].shape[0] /
                                     self.data[self.data['seller_country'] == country].shape[0] * 100
            }

        self.overall_stats = country_stats

        print("Overall statistics calculated.")
        return self

    def visualize_trends(self):
        """Create visualizations of brand and product trends by country."""
        if self.country_brand_stats is None or self.country_product_stats is None:
            raise ValueError("Brand and product analyses not performed. Please run analyze_brands_by_country() and analyze_products_by_country() first.")

        print("\n==== Creating Visualizations ====")

        # Set plot style
        sns.set(style=self.config['plots']['style'])

        # Create output directory if it doesn't exist
        os.makedirs(self.config['output']['figures_dir'], exist_ok=True)

        # 1. Top brands by country (heatmap)
        self._plot_brand_heatmap()

        # 2. Top product types by country (heatmap)
        self._plot_product_heatmap()

        # 3. Sales rates by country
        self._plot_sales_rates_by_country()

        # 4. Country-specific brand analysis (for top 5 countries)
        top_5_countries = self.data['seller_country'].value_counts().head(5).index
        for country in top_5_countries:
            self._plot_country_brand_analysis(country)
            self._plot_country_product_analysis(country)

        # 5. Price vs Sales Rate scatter plot
        self._plot_price_vs_sales_scatter()

        # 6. Overall country comparison
        self._plot_country_comparison()

        print("Visualizations created successfully.")
        return self

    def _plot_brand_heatmap(self):
        """Create a heatmap of brand sales rates by country."""
        # Get top countries and brands by listings
        top_countries = self.country_brand_stats['country'].value_counts().head(
            self.config['data']['top_n_countries']).index

        # Pivot the data for the heatmap
        pivot_data = self.country_brand_stats.pivot_table(
            index='country',
            columns='brand',
            values='sales_rate',
            aggfunc='mean'
        ).fillna(0)

        # Keep only top countries
        pivot_data = pivot_data.loc[pivot_data.index.isin(top_countries)]

        # For each country, keep only top N brands by sales rate
        top_brands_by_country = {}
        for country in pivot_data.index:
            top_brands = pivot_data.loc[country].nlargest(10).index
            top_brands_by_country[country] = top_brands

        # Get union of all top brands across countries
        all_top_brands = set()
        for brands in top_brands_by_country.values():
            all_top_brands.update(brands)

        # Filter pivot data to include only these brands
        pivot_data = pivot_data[list(all_top_brands)]

        # Create figure and axes
        fig, ax = plt.subplots(figsize=self.config['plots']['figsize_large'], dpi=self.config['plots']['dpi'])

        # Create custom colormap going from white to blue
        cmap = sns.color_palette("Blues", as_cmap=True)

        # Create heatmap
        sns.heatmap(
            pivot_data,
            annot=True,
            fmt='.1f',
            cmap=cmap,
            ax=ax,
            cbar_kws={'label': 'Sales Rate (%)'},
            linewidths=0.5
        )

        # Set labels and title
        ax.set_title('Brand Sales Rates by Country (%)', fontsize=self.config['plots']['title_fontsize'])
        ax.set_xlabel('Brand', fontsize=self.config['plots']['label_fontsize'])
        ax.set_ylabel('Country', fontsize=self.config['plots']['label_fontsize'])

        # Rotate x-axis labels
        plt.setp(ax.get_xticklabels(), rotation=45, ha='right')

        # Adjust layout and save
        fig.tight_layout()
        fig.savefig(os.path.join(self.config['output']['figures_dir'], 'brand_sales_rate_heatmap.png'))
        plt.close(fig)

    def _plot_product_heatmap(self):
        """Create a heatmap of product type sales rates by country."""
        # Get top countries and product types by listings
        top_countries = self.country_product_stats['country'].value_counts().head(
            self.config['data']['top_n_countries']).index

        # Pivot the data for the heatmap
        pivot_data = self.country_product_stats.pivot_table(
            index='country',
            columns='product_type',
            values='sales_rate',
            aggfunc='mean'
        ).fillna(0)

        # Keep only top countries
        pivot_data = pivot_data.loc[pivot_data.index.isin(top_countries)]

        # For each country, keep only top N product types by sales rate
        top_products_by_country = {}
        for country in pivot_data.index:
            top_products = pivot_data.loc[country].nlargest(8).index
            top_products_by_country[country] = top_products

        # Get union of all top product types across countries
        all_top_products = set()
        for products in top_products_by_country.values():
            all_top_products.update(products)

        # Filter pivot data to include only these product types
        pivot_data = pivot_data[list(all_top_products)]

        # Create figure and axes
        fig, ax = plt.subplots(figsize=self.config['plots']['figsize_large'], dpi=self.config['plots']['dpi'])

        # Create custom colormap going from white to green
        cmap = sns.color_palette("Greens", as_cmap=True)

        # Create heatmap
        sns.heatmap(
            pivot_data,
            annot=True,
            fmt='.1f',
            cmap=cmap,
            ax=ax,
            cbar_kws={'label': 'Sales Rate (%)'},
            linewidths=0.5
        )

        # Set labels and title
        ax.set_title('Product Type Sales Rates by Country (%)', fontsize=self.config['plots']['title_fontsize'])
        ax.set_xlabel('Product Type', fontsize=self.config['plots']['label_fontsize'])
        ax.set_ylabel('Country', fontsize=self.config['plots']['label_fontsize'])

        # Rotate x-axis labels
        plt.setp(ax.get_xticklabels(), rotation=45, ha='right')

        # Adjust layout and save
        fig.tight_layout()
        fig.savefig(os.path.join(self.config['output']['figures_dir'], 'product_type_sales_rate_heatmap.png'))
        plt.close(fig)

    def _plot_sales_rates_by_country(self):
        """Create a bar chart of overall sales rates by country."""
        # Extract overall sales rates by country
        if self.overall_stats is None:
            self.calculate_overall_statistics()

        country_sales_rates = {
            country: stats['overall_sales_rate']
            for country, stats in self.overall_stats.items()
        }

        # Convert to DataFrame and sort
        sales_rates_df = pd.DataFrame({
            'country': list(country_sales_rates.keys()),
            'sales_rate': list(country_sales_rates.values())
        }).sort_values('sales_rate', ascending=False)

        # Create figure and axes
        fig, ax = plt.subplots(figsize=self.config['plots']['figsize_medium'], dpi=self.config['plots']['dpi'])

        # Create bar chart
        bars = sns.barplot(
            data=sales_rates_df,
            x='country',
            y='sales_rate',
            ax=ax,
            palette='viridis'
        )

        # Add value labels on top of bars
        for i, bar in enumerate(bars.patches):
            ax.text(
                bar.get_x() + bar.get_width()/2,
                bar.get_height() + 0.1,
                f'{bar.get_height():.1f}%',
                ha='center',
                fontsize=self.config['plots']['annotation_fontsize']
            )

        # Set labels and title
        ax.set_title('Overall Sales Rate by Country', fontsize=self.config['plots']['title_fontsize'])
        ax.set_xlabel('Country', fontsize=self.config['plots']['label_fontsize'])
        ax.set_ylabel('Sales Rate (%)', fontsize=self.config['plots']['label_fontsize'])

        # Format y-axis as percentage
        ax.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=1))

        # Adjust layout and save
        fig.tight_layout()
        fig.savefig(os.path.join(self.config['output']['figures_dir'], 'sales_rate_by_country.png'))
        plt.close(fig)

    def _plot_country_brand_analysis(self, country):
        """Create a detailed analysis of brand performance for a specific country."""
        # Filter data for the specified country
        country_data = self.country_brand_stats[self.country_brand_stats['country'] == country]

        if country_data.empty:
            print(f"No data available for {country}")
            return

        # Sort by sales rate
        top_brands_by_rate = country_data.sort_values('sales_rate', ascending=False).head(10)

        # Create figure with GridSpec for custom layout
        fig = plt.figure(figsize=self.config['plots']['figsize_large'], dpi=self.config['plots']['dpi'])
        gs = GridSpec(2, 2, figure=fig, height_ratios=[1, 1], width_ratios=[2, 1])

        # Plot 1: Sales rate by brand (horizontal bar chart)
        ax1 = fig.add_subplot(gs[0, 0])
        bars1 = sns.barplot(
            data=top_brands_by_rate,
            y='brand',
            x='sales_rate',
            ax=ax1,
            palette='viridis',
            orient='h'
        )

        # Add value labels
        for i, bar in enumerate(bars1.patches):
            ax1.text(
                bar.get_width() + 0.2,
                bar.get_y() + bar.get_height()/2,
                f'{bar.get_width():.1f}%',
                va='center',
                fontsize=self.config['plots']['annotation_fontsize']
            )

        ax1.set_title(f'Top Brands by Sales Rate in {country}', fontsize=self.config['plots']['title_fontsize'])
        ax1.set_xlabel('Sales Rate (%)', fontsize=self.config['plots']['label_fontsize'])
        ax1.set_ylabel('Brand', fontsize=self.config['plots']['label_fontsize'])

        # Plot 2: Total sales by brand (horizontal bar chart)
        ax2 = fig.add_subplot(gs[1, 0])
        top_brands_by_sales = country_data.sort_values('sales', ascending=False).head(10)
        bars2 = sns.barplot(
            data=top_brands_by_sales,
            y='brand',
            x='sales',
            ax=ax2,
            palette='viridis',
            orient='h'
        )

        # Add value labels
        for i, bar in enumerate(bars2.patches):
            ax2.text(
                bar.get_width() + 0.2,
                bar.get_y() + bar.get_height()/2,
                f'{int(bar.get_width())}',
                va='center',
                fontsize=self.config['plots']['annotation_fontsize']
            )

        ax2.set_title(f'Top Brands by Total Sales in {country}', fontsize=self.config['plots']['title_fontsize'])
        ax2.set_xlabel('Number of Sales', fontsize=self.config['plots']['label_fontsize'])
        ax2.set_ylabel('Brand', fontsize=self.config['plots']['label_fontsize'])

        # Plot 3: Average price by brand (scatter plot with size indicating sales)
        ax3 = fig.add_subplot(gs[:, 1])
        scatter = ax3.scatter(
            country_data['sales_rate'],
            country_data['avg_price'],
            s=country_data['sales'] * 10,  # Size proportional to sales
            alpha=0.7,
            c=country_data['sales'],  # Color also based on sales
            cmap='viridis'
        )

        # Add brand labels to points
        for i, row in country_data.iterrows():
            if row['sales'] >= 3:  # Only label points with at least 3 sales
                ax3.annotate(
                    row['brand'],
                    (row['sales_rate'], row['avg_price']),
                    fontsize=8,
                    ha='center',
                    va='bottom',
                    xytext=(0, 5),
                    textcoords='offset points'
                )

        # Add a colorbar
        cbar = plt.colorbar(scatter, ax=ax3)
        cbar.set_label('Number of Sales', fontsize=self.config['plots']['label_fontsize'])

        ax3.set_title(f'Price vs. Sales Rate in {country}', fontsize=self.config['plots']['title_fontsize'])
        ax3.set_xlabel('Sales Rate (%)', fontsize=self.config['plots']['label_fontsize'])
        ax3.set_ylabel('Average Price (USD)', fontsize=self.config['plots']['label_fontsize'])

        # Set y-axis to log scale for better visualization of price distribution
        ax3.set_yscale('log')

        # Adjust layout and save
        plt.tight_layout()
        fig.savefig(os.path.join(self.config['output']['figures_dir'], f'{country}_brand_analysis.png'))
        plt.close(fig)

    def _plot_country_product_analysis(self, country):
        """Create a detailed analysis of product type performance for a specific country."""
        # Filter data for the specified country
        country_data = self.country_product_stats[self.country_product_stats['country'] == country]

        if country_data.empty:
            print(f"No product data available for {country}")
            return

        # Sort by sales rate
        top_products_by_rate = country_data.sort_values('sales_rate', ascending=False).head(10)

        # Create figure with GridSpec for custom layout
        fig = plt.figure(figsize=self.config['plots']['figsize_large'], dpi=self.config['plots']['dpi'])
        gs = GridSpec(2, 2, figure=fig, height_ratios=[1, 1], width_ratios=[2, 1])

        # Plot 1: Sales rate by product type (horizontal bar chart)
        ax1 = fig.add_subplot(gs[0, 0])
        bars1 = sns.barplot(
            data=top_products_by_rate,
            y='product_type',
            x='sales_rate',
            ax=ax1,
            palette='viridis',
            orient='h'
        )

        # Add value labels
        for i, bar in enumerate(bars1.patches):
            ax1.text(
                bar.get_width() + 0.2,
                bar.get_y() + bar.get_height()/2,
                f'{bar.get_width():.1f}%',
                va='center',
                fontsize=self.config['plots']['annotation_fontsize']
            )

        ax1.set_title(f'Top Product Types by Sales Rate in {country}', fontsize=self.config['plots']['title_fontsize'])
        ax1.set_xlabel('Sales Rate (%)', fontsize=self.config['plots']['label_fontsize'])
        ax1.set_ylabel('Product Type', fontsize=self.config['plots']['label_fontsize'])

        # Plot 2: Total sales by product type (horizontal bar chart)
        ax2 = fig.add_subplot(gs[1, 0])
        top_products_by_sales = country_data.sort_values('sales', ascending=False).head(10)
        bars2 = sns.barplot(
            data=top_products_by_sales,
            y='product_type',
            x='sales',
            ax=ax2,
            palette='viridis',
            orient='h'
        )

        # Add value labels
        for i, bar in enumerate(bars2.patches):
            ax2.text(
                bar.get_width() + 0.2,
                bar.get_y() + bar.get_height()/2,
                f'{int(bar.get_width())}',
                va='center',
                fontsize=self.config['plots']['annotation_fontsize']
            )

        ax2.set_title(f'Top Product Types by Total Sales in {country}', fontsize=self.config['plots']['title_fontsize'])
        ax2.set_xlabel('Number of Sales', fontsize=self.config['plots']['label_fontsize'])
        ax2.set_ylabel('Product Type', fontsize=self.config['plots']['label_fontsize'])

        # Plot 3: Average price by product type (scatter plot with size indicating sales)
        ax3 = fig.add_subplot(gs[:, 1])
        scatter = ax3.scatter(
            country_data['sales_rate'],
            country_data['avg_price'],
            s=country_data['sales'] * 10,  # Size proportional to sales
            alpha=0.7,
            c=country_data['sales'],  # Color also based on sales
            cmap='viridis'
        )

        # Add product type labels to points
        for i, row in country_data.iterrows():
            if row['sales'] >= 3:  # Only label points with at least 3 sales
                ax3.annotate(
                    row['product_type'],
                    (row['sales_rate'], row['avg_price']),
                    fontsize=8,
                    ha='center',
                    va='bottom',
                    xytext=(0, 5),
                    textcoords='offset points'
                )

        # Add a colorbar
        cbar = plt.colorbar(scatter, ax=ax3)
        cbar.set_label('Number of Sales', fontsize=self.config['plots']['label_fontsize'])

        ax3.set_title(f'Price vs. Sales Rate in {country}', fontsize=self.config['plots']['title_fontsize'])
        ax3.set_xlabel('Sales Rate (%)', fontsize=self.config['plots']['label_fontsize'])
        ax3.set_ylabel('Average Price (USD)', fontsize=self.config['plots']['label_fontsize'])

        # Set y-axis to log scale for better visualization of price distribution
        ax3.set_yscale('log')

        # Adjust layout and save
        plt.tight_layout()
        fig.savefig(os.path.join(self.config['output']['figures_dir'], f'{country}_product_analysis.png'))
        plt.close(fig)

    def _plot_price_vs_sales_scatter(self):
        """Create a scatter plot of price vs sales rate across all countries."""
        # Combine brand and product data
        brand_data = self.country_brand_stats.copy()
        brand_data['type'] = 'Brand'
        brand_data = brand_data.rename(columns={'brand': 'item'})

        product_data = self.country_product_stats.copy()
        product_data['type'] = 'Product Type'
        product_data = product_data.rename(columns={'product_type': 'item'})

        combined_data = pd.concat([
            brand_data[['country', 'item', 'type', 'sales_rate', 'avg_price', 'sales', 'listings']],
            product_data[['country', 'item', 'type', 'sales_rate', 'avg_price', 'sales', 'listings']]
        ])

        # Create figure
        fig, ax = plt.subplots(figsize=self.config['plots']['figsize_medium'], dpi=self.config['plots']['dpi'])

        # Separate by type for different markers
        for type_name, group in combined_data.groupby('type'):
            marker = 'o' if type_name == 'Brand' else '^'
            ax.scatter(
                group['sales_rate'],
                group['avg_price'],
                s=group['sales'] * 5,  # Size proportional to sales
                alpha=0.5,
                marker=marker,
                label=type_name
            )

        # Add a logarithmic trendline
        x = combined_data['sales_rate']
        y = combined_data['avg_price']

        try:
            z = np.polyfit(np.log(x + 1), y, 1)
            p = np.poly1d(z)
            x_trend = np.linspace(x.min(), x.max(), 100)
            ax.plot(x_trend, p(np.log(x_trend + 1)), 'r--', alpha=0.7)
        except:
            # Skip trendline if there's an error
            pass

        # Set labels and title
        ax.set_title('Price vs. Sales Rate Across All Countries', fontsize=self.config['plots']['title_fontsize'])
        ax.set_xlabel('Sales Rate (%)', fontsize=self.config['plots']['label_fontsize'])
        ax.set_ylabel('Average Price (USD)', fontsize=self.config['plots']['label_fontsize'])

        # Set y-axis to log scale
        ax.set_yscale('log')

        # Add legend
        ax.legend(title='Item Type')

        # Add grid
        ax.grid(True, alpha=0.3)

        # Adjust layout and save
        fig.tight_layout()
        fig.savefig(os.path.join(self.config['output']['figures_dir'], 'price_vs_sales_scatter.png'))
        plt.close(fig)

    def _plot_country_comparison(self):
        """Create a comparison of top performers across countries."""
        if self.overall_stats is None:
            self.calculate_overall_statistics()

        # Prepare data for the plot
        countries = list(self.overall_stats.keys())
        brands = []
        brand_rates = []
        products = []
        product_rates = []

        for country in countries:
            stats = self.overall_stats[country]
            if stats['top_brand_by_rate'] is not None:
                brands.append(stats['top_brand_by_rate']['brand'])
                brand_rates.append(stats['top_brand_by_rate']['sales_rate'])
            else:
                brands.append('N/A')
                brand_rates.append(0)

            if stats['top_product_by_rate'] is not None:
                products.append(stats['top_product_by_rate']['product_type'])
                product_rates.append(stats['top_product_by_rate']['sales_rate'])
            else:
                products.append('N/A')
                product_rates.append(0)

        # Create DataFrame
        comparison_data = pd.DataFrame({
            'Country': countries,
            'Top Brand': brands,
            'Brand Sales Rate': brand_rates,
            'Top Product': products,
            'Product Sales Rate': product_rates
        })

        # Sort by overall sales rate
        overall_rates = [self.overall_stats[country]['overall_sales_rate'] for country in countries]
        comparison_data['Overall Sales Rate'] = overall_rates
        comparison_data = comparison_data.sort_values('Overall Sales Rate', ascending=False)

        # Create figure
        fig, ax = plt.subplots(figsize=self.config['plots']['figsize_large'], dpi=self.config['plots']['dpi'])

        # Generate table
        table = ax.table(
            cellText=comparison_data.values,
            colLabels=comparison_data.columns,
            loc='center',
            cellLoc='center',
            colColours=['#f2f2f2'] * len(comparison_data.columns)
        )

        # Style the table
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1.2, 1.5)

        # Hide axes
        ax.axis('off')

        # Set title
        ax.set_title('Top Performers by Country', fontsize=self.config['plots']['title_fontsize'], pad=20)

        # Adjust layout and save
        fig.tight_layout()
        fig.savefig(os.path.join(self.config['output']['figures_dir'], 'country_comparison_table.png'))
        plt.close(fig)

    def generate_report(self):
        """Generate a summary report of best-selling brands and products by country."""
        if self.overall_stats is None:
            raise ValueError("Overall statistics not calculated. Please call calculate_overall_statistics() first.")

        print("\n==== Generating Report ====")

        # Create output directory if it doesn't exist
        os.makedirs(self.config['output']['data_dir'], exist_ok=True)

        # Save country-brand statistics to CSV
        if self.country_brand_stats is not None:
            self.country_brand_stats.to_csv(
                os.path.join(self.config['output']['data_dir'], 'country_brand_stats.csv'),
                index=False
            )

        # Save country-product statistics to CSV
        if self.country_product_stats is not None:
            self.country_product_stats.to_csv(
                os.path.join(self.config['output']['data_dir'], 'country_product_stats.csv'),
                index=False
            )

        # Print summary report
        print("\nSummary of Best-Selling Brands and Products by Country:")
        print("-------------------------------------------------------")

        for country, stats in self.overall_stats.items():
            print(f"\n{country}:")
            print(f"  Overall Sales Rate: {stats['overall_sales_rate']:.2f}%")

            if stats['top_brand_by_rate'] is not None:
                print(f"  Top Brand by Sales Rate: {stats['top_brand_by_rate']['brand']} ({stats['top_brand_by_rate']['sales_rate']:.2f}%)")

            if stats['top_brand_by_volume'] is not None:
                print(f"  Top Brand by Sales Volume: {stats['top_brand_by_volume']['brand']} ({int(stats['top_brand_by_volume']['sales'])} sales)")

            if stats['top_product_by_rate'] is not None:
                print(f"  Top Product by Sales Rate: {stats['top_product_by_rate']['product_type']} ({stats['top_product_by_rate']['sales_rate']:.2f}%)")

            if stats['top_product_by_volume'] is not None:
                print(f"  Top Product by Sales Volume: {stats['top_product_by_volume']['product_type']} ({int(stats['top_product_by_volume']['sales'])} sales)")

        # Create a summary table for all countries
        summary_data = []
        for country, stats in self.overall_stats.items():
            top_brand_rate = stats['top_brand_by_rate']['brand'] if stats['top_brand_by_rate'] is not None else 'N/A'
            top_brand_volume = stats['top_brand_by_volume']['brand'] if stats['top_brand_by_volume'] is not None else 'N/A'
            top_product_rate = stats['top_product_by_rate']['product_type'] if stats['top_product_by_rate'] is not None else 'N/A'
            top_product_volume = stats['top_product_by_volume']['product_type'] if stats['top_product_by_volume'] is not None else 'N/A'

            summary_data.append({
                'Country': country,
                'Overall Sales Rate (%)': round(stats['overall_sales_rate'], 2),
                'Top Brand by Rate': top_brand_rate,
                'Top Brand by Volume': top_brand_volume,
                'Top Product by Rate': top_product_rate,
                'Top Product by Volume': top_product_volume
            })

        # Convert to DataFrame and save
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_csv(
            os.path.join(self.config['output']['data_dir'], 'country_sales_summary.csv'),
            index=False
        )

        print(f"\nDetailed statistics saved to {self.config['output']['data_dir']}")
        return self

    def run_analysis(self, filepath):
        """Run the complete analysis pipeline."""
        return (
            self.load_data(filepath)
            .preprocess_data()
            .analyze_brands_by_country()
            .analyze_products_by_country()
            .calculate_overall_statistics()
            .visualize_trends()
            .generate_report()
        )


def main():
    """Main function to run the country brand analysis."""
    # Initialize analyzer
    analyzer = CountryBrandAnalyzer()

    # Run the analysis
    analyzer.run_analysis('/Users/samarthbasavarajannigeri/Downloads/vestiaire.csv')

    print("\n==== Analysis Complete ====")
    print("Best-selling brands and products by country have been identified and visualized.")


if __name__ == "__main__":
    main()

Loading data from /Users/samarthbasavarajannigeri/Downloads/vestiaire.csv...
Data loaded. Shape: (900514, 36)

==== Preprocessing Data ====
Preprocessing complete. Data shape: (900514, 36)

==== Analyzing Brands by Country ====
Analyzing data for 10 countries with at least 300 listings
Brand analysis complete. Analyzed 200 country-brand combinations.

==== Analyzing Product Types by Country ====
Product type analysis complete. Analyzed 150 country-product combinations.

==== Calculating Overall Statistics ====
Overall statistics calculated.

==== Creating Visualizations ====
Visualizations created successfully.

==== Generating Report ====

Summary of Best-Selling Brands and Products by Country:
-------------------------------------------------------

Italy:
  Overall Sales Rate: 1.52%
  Top Brand by Sales Rate: Saint Laurent (3.42%)
  Top Brand by Sales Volume: Gucci (249 sales)
  Top Product by Sales Rate: T-shirt (2.65%)
  Top Product by Sales Volume: T-shirt (166 sales)

France:
  